In [ ]:
import time
import datetime
import pandas as pd
import numpy as np
import pprint
import pyupbit

#import cryptocompare
#import tqdm
#import plotly.express as px
#import os
#import shutil
#import itertools

In [ ]:
import ccqt

bot_name = 'trade_LWVB2'

class TradeEnding(ccqt.trade.trade):
    def load_config(self, filename):
        super().load_config(self, filename)
        self.m_target_volatility = float(self.m_config['lwvb']['target_volatility'])



te = TradeEnding(bot_name)

te.load_config('trade_LWVB.ini')
te.prepare_slack('#lwvb-vn')
te.prepare_upbit()
te.check_upbit()
te.check_slack()

In [ ]:
# 처음 틀었을때: 다음 첫 개장할때까지 기다리기
# 오늘 개장시각 기준으로 다음첫개장시각 확인
now = te.get_current_time()
first_open_time = datetime.datetime(
    now.year, now.month, now.day, 
    te.m_config_open_time_hour, te.m_config_open_time_min, 0, tzinfo=datetime.timezone.utc)
if now < first_open_time:
    te.loginfo("Bot started before first open time {}".format(first_open_time))
else:
    te.loginfo("Bot started after {}".format(first_open_time))
    first_open_time += datetime.timedelta(days=1)
    te.loginfo("first open time would be {}".format(first_open_time))


# 다음첫개장할때까지 대기
te.loginfo("Before first trade, now, wait until {}".format(first_open_time))
te.standby_until(first_open_time)

te.print_slack_msg("now going to while loop...")


In [ ]:
next_open_time = first_open_time 
# 세팅모드 먼저 시작하도록 한다!


while True:
    # loop invariant:
    # 1. next_open_time = cur_open_time + 1day
    # (not now) 2. cur_open_time <= now < next_open_time 
    # 3. cur_open_time < clear_time < next_open_time
    now = te.get_current_time()

    # 하루의 끝이자 시작. 세팅 모드
    if next_open_time <= now:
        te.print_slack_msg("투자일 시작/종료 시각 도달, LWVB 초기 세팅 시작")
        cur_open_time = next_open_time
        next_open_time = next_open_time + datetime.timedelta(days=1)
        clear_time = next_open_time - datetime.timedelta(minutes=10)
        is_buy_completed = False
        is_sold_completed = False

        # MT여부 확인
        df = te.get_current_ohlcv_hour('BTC')
        hist_day = df.resample('1D', offset=datetime.timedelta(
            hours=te.m_config_open_time_hour,
            minutes=te.m_config_open_time_min)).agg(
        {
            'open': 'first',
            'close': 'last',
            'high': 'max',
            'low': 'min',
            'volume': 'sum',
            'value': 'sum',
        })
        if cur_open_time not in hist_day.index:
            te.logerror("cur_open_time not in hist_day.index")
            te.logerror(f'{cur_open_time=}')
            te.logerror(hist_day.index)

        hist_day['is_bull'] = True
        for open_dcount in [3, 5, 10]:
            colStr = 'maopen_{}'.format(open_dcount)
            hist_day[colStr] = hist_day['open'].shift(1).rolling(open_dcount).mean()
            hist_day['is_bull'] = hist_day['is_bull'] & (hist_day['open'].shift(1) > hist_day[colStr])

        te.print_slack_msg(str(hist_day[['open', 'high', 'low', 'close', 'is_bull']]))
        time.sleep(1)
        if not hist_day['is_bull'][-1]: 
            te.print_slack_msg("MT 조건 만족 안함! 오늘 하루 쉼")
            te.standby_until(next_open_time)
        
        # 기억하자:
          # hist_day 에서 마지막행은 오늘 데이터(시가만 유효)이고
          # 마지막 전 행이 어제 데이터다

        range_prev = hist_day['high'][-2] - hist_day['low'][-2]
        noise_prev = 1 - abs(hist_day['close'][-2] - hist_day['open'][-2]) / range_prev
        breakout_price = hist_day['open'][-1] + range_prev * noise_prev
        volatility = range_prev / hist_day['open'][-2]
        if volatility <= te.m_target_volatility:
            invest_ratio = 1
        else:
            invest_ratio = te.m_target_volatility / volatility

        today_open = hist_day['open'][-1]
        te.print_slack_msg(f'{range_prev=}, {noise_prev=}, {breakout_price=}, {volatility=}, {invest_ratio=}, {today_open=}')



    # loop invariant:
    # 1. next_open_time = cur_open_time + 1day
    # 2. cur_open_time <= now < next_open_time
    # 3. cur_open_time < clear_time < next_open_time
    # now loop invariant 2. holds
    
    # 하루의 종료에 다다름 정리 모드
    # 모두 팔고, 수익실현, 수익률계산하고, 현재 관리예금 업데이트
    if clear_time <= now :
        if is_sold_completed:
            te.print_slack_msg('매도 이미 완료, 다음 개장 시간 까지 대기 ...')
            te.standby_until(next_open_time)
        else:
            te.loginfo("Starting Selling...")
            btc_amount = upbit.get_balance("BTC")
            sellResult = upbit.sell_market_order("KRW-BTC", btc_amount)
            is_sold_completed = True

            if sellResult is None:
                te.logerror("critical error. sell order failed")
                te.print_slack_msg('매도 주문 실패' + f'{btc_amount=}')
            else:
                # check revenue_krw
                order_detail = upbit.get_order(sellResult['uuid'])
                revenue_krw = -float(order_detail['paid_fee'])
                for trade_detail in order_detail['trades']:
                    revenue_krw += float(trade_detail['funds'])
                revenue_krw = int(revenue_krw)
                budget_krw += revenue_krw
                te.print_slack_msg('매도 주문 완료' + f'{btc_amount=}, {revenue_krw=}, {budget_krw=}')
            
            te.standby_until(next_open_time)


    # 거래 모드
    if now < clear_time:
        if is_buy_completed:
            te.print_slack_msg('매수 이미 완료, 정리 시간 까지 대기 ...')
            te.standby_until(clear_time)
        else:
            curPrice = pyupbit.get_current_price('KRW-BTC')
            if breakout_price <= curPrice:
                te.loginfo("Starting Buying...")
                # 매수할 때는 수수료 고려한 금액을 넣어줘야한다.
                # 매도할 때는 암호화폐 총량만 넣어주면 됨 (매도 후 금액에서 수수료 공제되서 들어옴)
                buy_krw_wo_fee = budget_krw * invest_ratio / (1 + config_fee_rate_percent / 100)
                buyResult = upbit.buy_market_order('KRW-BTC', buy_krw_wo_fee)
                is_buy_completed = True
                if buyResult is None:
                    # 거래실패. 너무 낮은 금액으로 시도한 것일 수 있음
                    te.print_slack_msg('매수 주문 실패' + f'{buy_krw_wo_fee=}, {budget_krw=}, {invest_ratio=}')
                else:
                    # check cost krw
                    order_detail = upbit.get_order(buyResult['uuid'])
                    cost_krw = float(order_detail['paid_fee'])
                    for trade_detail in order_detail['trades']:
                        cost_krw += float(trade_detail['funds'])
                    cost_krw = int(cost_krw + 0.5)  # 반올림
                    budget_krw -= cost_krw
                    te.print_slack_msg('매수 주문 완료' + f'{cost_krw=}, {budget_krw=}')

    time.sleep(1)  # 1초에 한번 정도 API로 체크하는건 전혀 문제 없음
    te.loginfo("bot is running well ... now:{}".format(str(datetime.datetime.now())))